# Version control the parameter files

Here we learn a nice little trick to version-control the parameter files while tracking the simulation output directories for all the versions of the network.

In [1]:
import denest
import nest
import yaml
from pathlib import Path
from pprint import pprint

In [2]:
PARAMS_DIR = Path('./data/params')  # Where all the parameter files are
DATA_DIR = Path('./data/outputs')  # Where all the simulation output directories are

## 1. Turn ``PARAMS_DIR`` into a git directory

## 2. Make modifications to the parameters 

## 3. Run a simulation with the modified parameter files and track the output directory

### A. Generate a unique output directory name

For example, using the current date and time:

In [3]:
from datetime import datetime

output_dir_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

output_dir = Path(DATA_DIR)/output_dir_name
output_dir

PosixPath('data/outputs/2020-06-30_13:47:32')

### B. Create a symlink to the output directory in the version controlled `PARAMS_DIR`

In [4]:
def overwrite_dir_symlink(target, link_path):
    """Overwrite or create a symlink to a directory.
    
    ``target`` should be relative to ``link_path``"""
    import os, errno
    
    try:
        os.symlink(target, link_path)
    except OSError as e:
        if e.errno == errno.EEXIST:
            os.remove(link_path)
            os.symlink(target, link_path)
        else:
            raise e

In [5]:
# Needs path RELATIVE to symlink position 
overwrite_dir_symlink(Path('../outputs')/output_dir_name, PARAMS_DIR/'output')

### C. Run the simulation

In [6]:
denest.run(PARAMS_DIR/'tree_paths.yml', output_dir=output_dir)

2020-06-30 13:47:32,953 [denest] INFO: 

=== RUNNING SIMULATION ========================================================

2020-06-30 13:47:32,956 [denest] INFO: Loading parameter file paths from data/params/tree_paths.yml
2020-06-30 13:47:32,984 [denest] INFO: Finished loading parameter file paths
2020-06-30 13:47:32,989 [denest] INFO: Loading parameters files: 
['./network_tree.yml',
 './simulation.yml',
 './session_models.yml',
 './kernel.yml']
2020-06-30 13:47:33,080 [denest] INFO: Initializing simulation...
2020-06-30 13:47:33,113 [denest.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input'}
2020-06-30 13:47:33,115 [denest.simulation] INFO: Initializing NEST kernel and seeds...
2020-06-30 13:47:33,116 [denest.simulation] INFO:   Resetting NEST kernel...
2020-06-30 13:47:33,279 [denest.simulation] INFO:   Setting NEST kernel status...
2020-06-30 13:47:33,299 [denest.simulation] INFO:     Calling `nest.SetKernelStatus

2020-06-30 13:47:34,902 [denest.session] INFO: Finished initializing session

2020-06-30 13:47:34,904 [denest.session] INFO: Running session '01_3_spikes' for 100 ms
2020-06-30 13:47:35,115 [denest.session] INFO: Finished running session
2020-06-30 13:47:35,116 [denest.session] INFO: Session '01_3_spikes' virtual running time: 100 ms
2020-06-30 13:47:35,117 [denest.session] INFO: Session '01_3_spikes' real running time: 0h:00m:00s
2020-06-30 13:47:35,118 [denest.simulation] INFO: Done running session '01_3_spikes'
2020-06-30 13:47:35,120 [denest.simulation] INFO: Running session: '02_2_spikes'...
2020-06-30 13:47:35,125 [denest.session] INFO: Initializing session...
2020-06-30 13:47:35,126 [denest.session] INFO: Setting `origin` flag to `200.0` for all stimulation devices in ``InputLayers`` for session `02_2_spikes`
2020-06-30 13:47:35,159 [denest.utils.validation] INFO: Object `Unit changes dictionary`: params: using default value for optional parameters:
{'change_type': 'constant', '

The simulation output was saved in the unique output directory created.  

In [7]:
print(output_dir)
!ls {output_dir}

data/outputs/2020-06-30_13:47:32
data               parameter_tree.yml session_times.yml  versions.txt


The `PARAMS_DIR/output` symlink points to the actual output directory.

In [8]:
!ls {PARAMS_DIR/'output/'}

data               parameter_tree.yml session_times.yml  versions.txt


## 4. Commit the `PARAMS_DIR` with the updated params and `output` symlimk